In [ ]:
import lips
from lips import Particles
import numpy
from syngular import Field
from pyadic.finite_field import ModP, rationalise

from bgtrees.currents import J_μ, another_j
from bgtrees.finite_gpufields.finite_fields_tf import FiniteField
from bgtrees.finite_gpufields.operations import ff_dot_product
from bgtrees.metric_and_verticies import η
from bgtrees.settings import settings
from bgtrees.states import εm, εp

lips.spinor_convention = "asymmetric"
chosenP = 2**31 - 19
NTEST = 25
settings.run_tf_eagerly()

In [ ]:
def _generate_input(chosen_field, helconf, n=25):
    """Generate the momentum and polarization arrays using lips
    Returns the momentum and polarization as numpy array and the
    list of lips particle_lists
    """
    lmoms = []
    lpols = []
    lPs = []

    for seed in range(n):
        particle_list = Particles(len(helconf), field=chosen_field, seed=seed)
        particle_list.helconf = helconf

        # Prepare hte momentum array
        lm = []
        for oParticle in particle_list:
            lm.append(numpy.block([oParticle.four_mom, numpy.array([0, 0, 0, 0])]))
        lmoms.append(numpy.array(lm))

        # Prepare the polarization array
        lp = []
        for index, helconf_index in enumerate(helconf):
            if helconf_index == "p":
                tmp = [εp(particle_list, index + 1), numpy.array([0, 0, 0, 0])]
            elif helconf_index == "m":
                tmp = [εm(particle_list, index + 1), numpy.array([0, 0, 0, 0])]
            elif helconf_index == "x":
                tmp = [numpy.array([0, 0, 0, 0, 0, 0, 1, 0])]
            else:
                tmp = None
            lp.append(numpy.block(tmp))

        lpols.append(numpy.array(lp))
        lPs.append(particle_list)

    lmoms = numpy.array(lmoms)
    lpols = numpy.array(lpols)

    return lmoms, lpols, lPs

In [ ]:
chosen_field = Field("finite field", chosenP, 1)
helconf = "ppmmm"
verbose = False

In [ ]:
lmoms, lpols, lPs = _generate_input(chosen_field, helconf)

In [ ]:
D = lmoms.shape[-1]
D

In [ ]:
numpy.all(numpy.einsum("rim->rm", lmoms) == 0)

In [ ]:
numpy.all(numpy.einsum("rim,mn,rin->ri", lmoms, η(D), lpols) == 0)

In [ ]:
res_cpu = numpy.einsum("rm,rm->r", lpols[:, 0], J_μ(lmoms[:, 1:], lpols[:, 1:], put_propagator=False, verbose=verbose))
res_cpu

In [ ]:
prev_setting = settings.use_gpu
settings.use_gpu = True

# Now make it into finite Finite Fields containers
def _make_to_container(array_of_arrays, p):
    """Make any array of arrays or list of list into a finite field container"""
    return FiniteField(array_of_arrays.astype(int), p)

ff_moms = _make_to_container(lmoms, chosenP)
ff_pols = _make_to_container(lpols, chosenP)

res_gpu = another_j(ff_moms[:, 1:], ff_pols[:, 1:], put_propagator=False, verbose=verbose)
res_gpu = ff_dot_product(ff_pols[:, 0], res_gpu)

settings.use_gpu = prev_setting
res_gpu

In [ ]:
res_cpu = numpy.einsum("rm,rm->r", lpols[:, 0], J_μ(lmoms[:, 1:], lpols[:, 1:], put_propagator=False, verbose=verbose))
res_cpu

In [ ]:
res_analytic = numpy.array([oPs("(-16[12]^4)/([12][23][34][45][51])") for oPs in lPs])
res_analytic

In [ ]:
assert numpy.all(res_gpu.values.numpy().astype(int) == res_cpu.astype(int))
assert numpy.all(res_cpu.astype(int) == res_analytic.astype(int))